<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/GS_v0_2_con_vocab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
base=pd.read_csv('/content/drive/MyDrive/base_formateada.csv')

<ipython-input-2-dc6dd172c19c>:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  base=pd.read_csv('/content/drive/MyDrive/base_formateada.csv')


In [3]:
import pickle

with open('/content/drive/MyDrive/vocabulary.voc', 'rb') as f:
    inter = pickle.load(f)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(2,2) ,vocabulary=inter)
matrix = vect.fit_transform(base['tokens'])

In [5]:
len(vect.vocabulary_)

270230

In [6]:
# Esta celda tarda unos 40 minutos en correr
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=30, max_iter=50, learning_method='batch', learning_offset=50.,random_state=0)  
lda.fit(matrix)

LatentDirichletAllocation(learning_offset=50.0, max_iter=50, n_components=30,
                          random_state=0)

In [7]:
import numpy as np
from numpy import argsort
import statistics
from statistics import mode
from scipy.special import logsumexp
import scipy as sp

# Topics por weights

In [8]:
disc_weights=matrix*lda.components_.transpose()

In [9]:
base['Topic_weight']=disc_weights.argmax(axis=1)+1

In [10]:
base.fecha=pd.to_datetime(base.fecha, infer_datetime_format=True)

In [16]:
# Por LEGISLATURA por WEIGHTS
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas: 
  base_legislatura=base[base.legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_weight'].value_counts().sort_index()/base_legislatura['Topic_weight'].value_counts().sort_index().sum()
topics_por_legislatura.to_csv('topics_por_legislatura_por_weights.csv')

In [17]:
# Por AÑO por weights
topics_por_año=pd.DataFrame()
for x in range (1979, 2020): 
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  topics_por_año[x]=base_year['Topic_weight'].value_counts().sort_index()/base_year['Topic_weight'].value_counts().sort_index().sum()
topics_por_año.to_csv('topics_por_año_por_weights.csv')

# FREX

In [18]:
def frex(mod, w=0.7):
    log_beta = np.log(mod.components_)
    log_exclusivity = log_beta - logsumexp(log_beta, axis=0)
    exclusivity_ecdf = np.apply_along_axis(ecdf, 1, log_exclusivity)
    freq_ecdf = np.apply_along_axis(ecdf, 1, log_beta)
    out = 1. / (w / exclusivity_ecdf + (1 - w) / freq_ecdf)
    return out

def ecdf(arr):
    return sp.stats.rankdata(arr, method='max') / arr.size

def t_FREX(row):
  return frex_df[matrix[row['id']].argmax()].argsort()[9]

In [19]:
frex_df=pd.DataFrame(frex(lda))

In [20]:
# Se crea una columna con el topic que se asigna por frex a cada discurso de la base
base['id']=range(0,334421)
base['Topic_FREX']=base.apply (lambda row: t_FREX(row), axis=1)+1

In [21]:
# Por LEGISLATURA por FREX
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas: 
  base_legislatura=base[base.legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_FREX'].value_counts().sort_index()/base_legislatura['Topic_FREX'].value_counts().sort_index().sum()
topics_por_legislatura.to_csv('topics_por_legislatura_por_frex.csv')

In [22]:
# Por AÑO por FREX
topics_por_año=pd.DataFrame()
for x in range (1979, 2020): 
  base_year=base[pd.DatetimeIndex(base.fecha).year==x]
  topics_por_año[x]=base_year['Topic_FREX'].value_counts().sort_index()/base_year['Topic_FREX'].value_counts().sort_index().sum()
topics_por_año.to_csv('topics_por_año_por_frex.csv')

# Bigramas por weights

In [41]:
bigramas=pd.DataFrame()
feature_names=vect.get_feature_names_out()
no_top_words=20
for topic_idx, topic in enumerate(lda.components_):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [42]:
bigramas.to_csv('bigramas_por_weights_30topics.csv')

# Bigramas por frex

In [43]:
"Topic %d:" % (topic_idx + 1)
for topic_idx, topic in enumerate(frex(lda)):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [44]:
bigramas.to_csv('bigramas_por_frex_30topics.csv')

# Bigramas para 10 Topics

In [36]:
# Esta celda tarda unos 25 minutos en correr
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, max_iter=50, learning_method='batch', learning_offset=50.,random_state=0)  
lda.fit(matrix)

LatentDirichletAllocation(learning_offset=50.0, max_iter=50, random_state=0)

In [37]:
bigramas=pd.DataFrame()
feature_names=vect.get_feature_names_out()
no_top_words=20
for topic_idx, topic in enumerate(lda.components_):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [38]:
bigramas.to_csv('bigramas_por_weights_10topics.csv')

In [39]:
"Topic %d:" % (topic_idx + 1)
for topic_idx, topic in enumerate(frex(lda)):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [40]:
bigramas.to_csv('bigramas_por_frex_10topics.csv')